In [1]:
import os
import numpy as np
import pandas as pd
import json

from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
#where the json files are extracted using the script provided by semeval
root = "C:\\Users\\rishi\\Documents\\Python Workspace\\data-extraction-semeval-v2\\output_dir"

In [3]:
json_files = set()

for subdir, dirs, files in os.walk(root):
    for f in files:
        if f.endswith(".html"):
            continue

        json_files.add(f[0:-5])

len(json_files)

9004

In [14]:
df_train = pd.read_csv('../dataset/train_v2.csv', index_col='pair_id')
df_master = pd.DataFrame(columns=['lang', 'link', 'title', 'text'], index=json_files)
df_master.index.name = 'id'

In [15]:
df_train['id1'] = df_train.index.to_series().apply(lambda x : x.split('_')[0])
df_train['id2'] = df_train.index.to_series().apply(lambda x : x.split('_')[1])
df_train

,url1_lang,url2_lang,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone,id1,id2
pair_id,,,,,,,,,,,,,,,
1484084337_1484110209,en,en,https://www.washingtonpost.com/local/virginia-...,https://www.washingtonpost.com/world/the_ameri...,https://web.archive.org/web/www.washingtonpost...,https://web.archive.org/web/www.washingtonpost...,4.0,4.000000,1.000000,4.000000,4.000000,1.666667,2.000000,1484084337,1484110209
1484396422_1483924666,en,en,https://www.stlucianewsonline.com/guyana-three...,https://www.thestar.com/news/world/europe/2020...,https://web.archive.org/web/www.stlucianewsonl...,https://web.archive.org/web/www.thestar.com/ne...,4.0,4.000000,1.000000,4.000000,3.666667,1.666667,1.333333,1484396422,1483924666
1484698254_1483758694,en,en,https://www.teaparty.org/trump-brings-in-2020-...,https://www.timesofisrael.com/trump-says-he-do...,https://web.archive.org/web/www.teaparty.org/t...,https://web.archive.org/web/www.timesofisrael....,1.0,2.000000,1.000000,2.333333,2.333333,1.000000,1.333333,1484698254,1483758694
1576314516_1576455088,en,en,https://gadgets.ndtv.com/apps/news/zomato-uber...,https://gadgets.ndtv.com/internet/news/indian-...,https://web.archive.org/web/gadgets.ndtv.com/a...,https://web.archive.org/web/gadgets.ndtv.com/i...,1.0,2.333333,2.666667,1.666667,2.000000,1.666667,1.666667,1576314516,1576455088
1484036253_1483894099,en,en,https://news.yahoo.com/india-approves-third-mo...,https://www.channelnewsasia.com/news/asia/indi...,https://web.archive.org/web/news.yahoo.com/ind...,https://web.archive.org/web/www.channelnewsasi...,1.0,1.250000,1.000000,1.250000,1.250000,1.000000,1.000000,1484036253,1483894099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586195445_1598778991,tr,tr,http://www.haberler.com/bm-aden-de-2-bini-aski...,https://www.haberler.com/bm-den-yemen-de-kadin...,https://web.archive.org/web/www.haberler.com/b...,https://web.archive.org/web/www.haberler.com/b...,1.0,2.000000,2.000000,4.000000,3.000000,1.000000,1.000000,1586195445,1598778991
1590915424_1590940388,tr,tr,https://www.haberler.com/kovid-19-dan-dolayi-l...,https://www.aksam.com.tr/spor/fabio-capello-ko...,https://web.archive.org/web/www.haberler.com/k...,https://web.archive.org/web/www.aksam.com.tr/s...,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1590915424,1590940388
1526157103_1492737005,tr,tr,http://www.samanyoluhaber.com/saray-da-care-ol...,https://www.fotomac.com.tr/besiktas/2020/01/12...,https://web.archive.org/web/www.samanyoluhaber...,https://web.archive.org/web/www.fotomac.com.tr...,1.0,2.000000,3.000000,4.000000,3.000000,1.000000,2.000000,1526157103,1492737005


In [16]:
def parse_json(subdir, f):

    loaded = json.load(open(os.path.join(subdir, f), encoding = 'utf-8'))

    article_id = f[0:-5]
    print(article_id)

    #other data like keywords and metadata could be extracted from json as well
    select1 = df_train.loc[df_train['id1'] == article_id]
    select2 = df_train.loc[df_train['id2'] == article_id]

    lang = select1.url1_lang[0] if not select1.empty else select2.url2_lang[0]
    url = select1.link1[0] if not select1.empty else select2.link2[0]

    df_master.loc[article_id, 'lang'] = lang
    df_master.loc[article_id, 'link'] = url
    df_master.loc[article_id, 'text'] = loaded['text']
    df_master.loc[article_id, 'meta_description'] = loaded['meta_description']

    try:
        df_master.loc[article_id, 'description'] = loaded['meta_data']["og"]["description"]
    except Exception as e:
        pass
    
    # try:
    #     df_master.loc[article_id, 'description'] = loaded['meta_data']["og"]["description"]
    # except Exception as e:
    #     pass
    
    df_master.loc[article_id, 'title'] = loaded['title']

    # if (len(loaded['meta_keywords']) == 0 or loaded['meta_keywords'][0] == ""):
    #     print(article_id)

In [16]:
for subdir, dirs, files in os.walk(root):
    for f in files:
        if f.endswith(".html"):
            continue
        
        parse_json(subdir, f)

1483733400
1483747000
1483803400
1483806200
1483806300
1483816300
1483830300
1483867300
1483971700
1483981600
1483988400
1483989600
1484009700
1484011800
1484012400
1484024600
1484031600
1484034100
1484036300
1484040200
1484040300
1484069700
1484089500
1484112800
1484159400
1484183400
1484188300
1484188700
1484189100
1484189300
1484190300
1484216200
1484255400
1484261400
1484301400
1484390300
1484404000
1484573400
1484698300
1484705100
1484906200
1484938600
1485180100
1485674200
1486823000
1493005700
1493645200
1494936800
1495768900
1497493200
1498603200
1504077800
1514219000
1520758400
1536337900
1557317500
1558542900
1558547800
1560364300
1564832000
1483772201
1483777901
1483836201
1483836501
1483841001
1483845101
1483870701
1483909901
1483910801
1483930001
1483931201
1483933201
1483981401
1484010801
1484030601
1484032701
1484033801
1484034001
1484034301
1484035601
1484036101
1484037501
1484049501
1484072601
1484113101
1484126601
1484146501
1484146701
1484176201
1484178601
1484188201

In [17]:
df_master.replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [18]:
df_master

,lang,link,title,text,meta_description,description
id,,,,,,
1542549977,de,https://uncut-news.ch/2020/03/09/ditib-und-die...,DITIB und die türkische Regierung unterwandern...,….das Transatlantischen Mediennetzwerk will de...,"unzensierte infos,uncut, infos, wichtige infos...","unzensierte infos,uncut, infos, wichtige infos..."
1484011708,en,https://tradingeconomics.com/euro-area/governm...,Euro Area Government Debt to GDP,Euro Area Government Debt to GDP\n\nThe Govern...,The Government Debt in the Euro Area was last ...,NaN
1516931045,en,https://www.freemalaysiatoday.com/category/wor...,Vietnam culls tens of thousands birds to conta...,HANOI: Vietnam has so far this year culled ten...,Authorities have also disinfected farms and ba...,Authorities have also disinfected farms and ba...
1483925044,pl,http://fronda.pl/a/jerzy-bukowski-wszechobecny...,Jerzy Bukowski: Wszechobecny Tusk,"Nie wystartuje w wyborach prezydenckich, nie z...",NaN,NaN
1484133398,en,https://www.bizpacreview.com/2020/01/01/rosean...,Roseanne Barr praises Trump for protecting Jew...,Roseanne Barr defended President Trump’s relat...,Actress and comedian Roseanne Barr praised Pre...,Actress and comedian Roseanne Barr praised Pre...
...,...,...,...,...,...,...
1484146010,en,https://www.kjrh.com/lifestyle/single-mother-i...,"Single mother in Michigan gets $2,020 tip on $...",A single mom with three children and one on th...,A single mother from Detroit moved Up North to...,A single mother from Detroit moved Up North to...
1484181227,en,https://detroit.cbslocal.com/2019/12/30/33-yea...,"33-Year-Old Man Shot, Killed During Altercatio...",New New American Spot Highlands Debuts In Down...,Police say a man was shot and killed during an...,Police say a man was shot and killed during an...
1484038319,pl,https://pikio.pl/rodzice-0101202-bm-z-dumy/,Brutalny wyczyn 8-letniej dziewczynki. Rodzice...,Rodzice są bardzo dumni z wyczynu swojej 8-let...,Rodzice 8-latki są z niej wyjątkowo dumni. Osi...,Rodzice nie widzą nic złego w brutalnym wyczyn...


In [19]:
df_master.title.isna().sum(), df_master.text.isna().sum(), df_master.meta_description.isna().sum(), df_master.description.isna().sum() 

(37, 31, 1024, 427)

In [20]:
df_master[df_master.meta_description.isna() & (df_master.description.isna())]

,lang,link,title,text,meta_description,description
id,,,,,,
1483925044,pl,http://fronda.pl/a/jerzy-bukowski-wszechobecny...,Jerzy Bukowski: Wszechobecny Tusk,"Nie wystartuje w wyborach prezydenckich, nie z...",NaN,NaN
1506515410,es,https://www.elclarinete.com.mx/los-ovnis-son-r...,“Los ovnis son reales”: La confirmación oficia...,Redacción\n\nEU.-Los ovnis siempre han sido ob...,NaN,NaN
1484284502,en,http://www.assamtribune.com/scripts/detailsnew...,The Assam Tribune Online,On the prevailing situation along the border w...,NaN,NaN
1525899670,es,https://diariolalibertad.com/sitio/2020/02/19/...,Ocho muertos y cinco heridos graves en dos tir...,Al menos ocho personas murieron en dos tiroteo...,NaN,NaN
1491944991,es,http://entrelineas.com.mx/mundo/soleimani-plan...,“Soleimani planeaba ataque contra embajadas de...,"El presidente Estados Unidos, Donald Trump, di...",NaN,NaN
...,...,...,...,...,...,...
1484434933,en,https://electionlawblog.org/?p=108630,Federal court issues preliminary injunction ag...,"The law, enacted over the Governor’s veto, usu...",NaN,NaN
1484231118,en,http://thenewsdoctors.com/post-iraq-war-us-int...,Post-Iraq war US intel report predicting 2020 ...,A 15-year-old intel report predicting the worl...,NaN,NaN
1484248149,en,https://www.sxm-talks.com/st-martin-news-netwo...,Honorable Prime Minister Silveria E. Jacobs Ne...,PHILIPSBURG:— Whether you will be heading to a...,NaN,NaN


In [57]:
pd.isnull(df_master.loc['1484028040', 'description'])

True

In [24]:
df_master["extra_text"] = df_master.progress_apply(
    lambda row: row['description'] if row['description'] != np.nan else row['meta_description'],
    axis = 1
)

  0%|          | 0/5245 [00:00<?, ?it/s]

In [28]:
for index, row in df_master.iterrows():
    if not pd.isna(row['description']):
        df_master.loc[index,'extra_text'] = row['description']
        # print(row)
    if not pd.isna(row['meta_description']):
        df_master.loc[index,'extra_text'] = row['meta_description']
        # print(row)


In [29]:
df_master

,lang,link,title,text,meta_description,description,extra_text
id,,,,,,,
1542549977,de,https://uncut-news.ch/2020/03/09/ditib-und-die...,DITIB und die türkische Regierung unterwandern...,….das Transatlantischen Mediennetzwerk will de...,"unzensierte infos,uncut, infos, wichtige infos...","unzensierte infos,uncut, infos, wichtige infos...","unzensierte infos,uncut, infos, wichtige infos..."
1484011708,en,https://tradingeconomics.com/euro-area/governm...,Euro Area Government Debt to GDP,Euro Area Government Debt to GDP\n\nThe Govern...,The Government Debt in the Euro Area was last ...,NaN,The Government Debt in the Euro Area was last ...
1516931045,en,https://www.freemalaysiatoday.com/category/wor...,Vietnam culls tens of thousands birds to conta...,HANOI: Vietnam has so far this year culled ten...,Authorities have also disinfected farms and ba...,Authorities have also disinfected farms and ba...,Authorities have also disinfected farms and ba...
1483925044,pl,http://fronda.pl/a/jerzy-bukowski-wszechobecny...,Jerzy Bukowski: Wszechobecny Tusk,"Nie wystartuje w wyborach prezydenckich, nie z...",NaN,NaN,NaN
1484133398,en,https://www.bizpacreview.com/2020/01/01/rosean...,Roseanne Barr praises Trump for protecting Jew...,Roseanne Barr defended President Trump’s relat...,Actress and comedian Roseanne Barr praised Pre...,Actress and comedian Roseanne Barr praised Pre...,Actress and comedian Roseanne Barr praised Pre...
...,...,...,...,...,...,...,...
1484146010,en,https://www.kjrh.com/lifestyle/single-mother-i...,"Single mother in Michigan gets $2,020 tip on $...",A single mom with three children and one on th...,A single mother from Detroit moved Up North to...,A single mother from Detroit moved Up North to...,A single mother from Detroit moved Up North to...
1484181227,en,https://detroit.cbslocal.com/2019/12/30/33-yea...,"33-Year-Old Man Shot, Killed During Altercatio...",New New American Spot Highlands Debuts In Down...,Police say a man was shot and killed during an...,Police say a man was shot and killed during an...,Police say a man was shot and killed during an...
1484038319,pl,https://pikio.pl/rodzice-0101202-bm-z-dumy/,Brutalny wyczyn 8-letniej dziewczynki. Rodzice...,Rodzice są bardzo dumni z wyczynu swojej 8-let...,Rodzice 8-latki są z niej wyjątkowo dumni. Osi...,Rodzice nie widzą nic złego w brutalnym wyczyn...,Rodzice 8-latki są z niej wyjątkowo dumni. Osi...


In [30]:
df_master['extra_text'].isna().sum()

237

In [31]:
df_master.drop(columns=["meta_description", "description"], inplace=True)

In [17]:
text_missing_ids = set(df_master.loc[df_master.text.isna()].index)

In [18]:
total = set(df_train.index.to_series().apply(lambda id : id.split('_')[0])).union(set(df_train.index.to_series().apply(lambda id : id.split('_')[1])))

In [19]:
not_present = total.symmetric_difference(json_files)

In [20]:
print("{} - {} = {} json files missing".format(len(total), len(json_files), len(not_present)))

9476 - 9004 = 472 json files missing


{'1483744049',
 '1483747000',
 '1483767490',
 '1483782363',
 '1483804274',
 '1483806257',
 '1483806365',
 '1483806495',
 '1483807252',
 '1483808624',
 '1483814471',
 '1483818682',
 '1483820338',
 '1483822410',
 '1483829594',
 '1483843306',
 '1483844855',
 '1483849920',
 '1483855112',
 '1483855447',
 '1483859003',
 '1483867584',
 '1483880574',
 '1483885933',
 '1483891773',
 '1483899162',
 '1483901660',
 '1483907828',
 '1483921648',
 '1483922990',
 '1483922997',
 '1483925832',
 '1483936557',
 '1483937746',
 '1483937983',
 '1483939960',
 '1483943085',
 '1483945069',
 '1483950498',
 '1483953562',
 '1483962884',
 '1483963076',
 '1483963364',
 '1483973584',
 '1483978026',
 '1483979746',
 '1483984595',
 '1483992754',
 '1483996182',
 '1483998251',
 '1483999027',
 '1483999729',
 '1484003508',
 '1484004794',
 '1484004798',
 '1484005279',
 '1484005288',
 '1484005594',
 '1484007287',
 '1484007530',
 '1484007543',
 '1484007787',
 '1484008055',
 '1484010218',
 '1484010524',
 '1484010964',
 '14840111

In [33]:
df_master.to_csv('../dataset/json_text_extract_v1_desc.csv')

In [2]:
import nltk

In [9]:
nltk.sent_tokenize('الله أكبر! لقد ربحت يانصيب')

['الله أكبر!', 'لقد ربحت يانصيب']